In [1]:
import scipy
import random
import portpy.photon as pp
import numpy as np
from cvxpy import *
from scipy import sparse
import sklearn.metrics
import time
import sys
import cvxpy as cp

In [2]:
data_dir = r'../data'
data = pp.DataExplorer(data_dir=data_dir)
patient_id = 'Lung_Phantom_Patient_1'
data.patient_id = patient_id
ct = pp.CT(data)
structs = pp.Structures(data)
beams = pp.Beams(data)
opt_params = data.load_config_opt_params(protocol_name='Lung_2Gy_30Fx')
structs.create_opt_structures(opt_params)
inf_matrix_sparse = pp.InfluenceMatrix(ct=ct, structs=structs, beams=beams)
protocol_name = 'Lung_2Gy_30Fx'
clinical_criteria = pp.ClinicalCriteria(data, protocol_name)
plan_sparse = pp.Plan(ct, structs, beams, inf_matrix_sparse, clinical_criteria)
opt = pp.Optimization(plan_sparse, opt_params=opt_params)
opt.create_cvxpy_problem()

sol_sparse = opt.solve(solver='MOSEK', verbose=True)

dose_sparse_1d = plan_sparse.inf_matrix.A @ (sol_sparse['optimal_intensity'] * plan_sparse.get_num_of_fractions())
x_sparse =sol_sparse['optimal_intensity'] * plan_sparse.get_num_of_fractions()


creating rinds.. This step may take some time due to dilation
rinds created!!
Creating BEV..
Loading sparse influence matrix...
Done
Objective Start
Objective done
Constraints Start
Structure ESOPHAGUS not available!
Structure ESOPHAGUS not available!
Constraints done
                                     CVXPY                                     
                                     v1.3.1                                    
(CVXPY) Oct 27 04:13:06 PM: Your problem has 1946 variables, 14 constraints, and 0 parameters.
(CVXPY) Oct 27 04:13:06 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Oct 27 04:13:06 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Oct 27 04:13:06 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation       

(CVXPY) Oct 27 04:13:17 PM: 21  3.3e-07  1.1e-06  3.1e-11  1.00e+00   4.204653231e+01   4.204653206e+01   2.3e-09  5.05  
(CVXPY) Oct 27 04:13:17 PM: 22  9.5e-08  3.1e-07  4.8e-12  9.56e-01   4.204568395e+01   4.204568387e+01   6.6e-10  5.19  
(CVXPY) Oct 27 04:13:17 PM: 23  7.2e-08  7.8e-08  6.2e-13  1.01e+00   4.204544808e+01   4.204544806e+01   1.7e-10  5.31  
(CVXPY) Oct 27 04:13:17 PM: Optimizer terminated. Time: 5.48    
(CVXPY) Oct 27 04:13:17 PM: 
(CVXPY) Oct 27 04:13:17 PM: 
(CVXPY) Oct 27 04:13:17 PM: Interior-point solution summary
(CVXPY) Oct 27 04:13:17 PM:   Problem status  : PRIMAL_AND_DUAL_FEASIBLE
(CVXPY) Oct 27 04:13:17 PM:   Solution status : OPTIMAL
(CVXPY) Oct 27 04:13:17 PM:   Primal.  obj: 4.2045448085e+01    nrm: 1e+02    Viol.  con: 6e-06    var: 2e-06    cones: 0e+00  
(CVXPY) Oct 27 04:13:17 PM:   Dual.    obj: 4.2045448063e+01    nrm: 6e+03    Viol.  con: 0e+00    var: 8e-07    cones: 0e+00  
------------------------------------------------------------------

In [30]:
start_time = time.time()

#sol_sparse = opt.solve(solver='MOSEK', verbose=True)
dose_sparse_1d = plan_sparse.inf_matrix.A @ (sol_sparse['optimal_intensity'] * plan_sparse.get_num_of_fractions())
plan_sparse = pp.Plan(ct, structs, beams, inf_matrix_sparse, clinical_criteria)
A_sparse = plan_sparse.inf_matrix.A

end_time = time.time()
total_execution_time = end_time - start_time
print(f"زمان ساخت ماتریس پراکنده\n: {total_execution_time} ثانیه")



زمان ساخت ماتریس پراکنده
: 0.003991365432739258 ثانیه


In [11]:
start_time = time.time()

[U,S,V]=scipy.sparse.linalg.svds(A_sparse,k=541)

rank = len([s for s in S if s > 1e-12])       #بدون تجزیه، چطور رتبه رو بدونم؟
print("رتبه ماتریس:", rank)


end_time = time.time()
total_execution_time = end_time - start_time
print(f"زمان کل اجرا: {total_execution_time} ثانیه")

رتبه ماتریس: 541
زمان کل اجرا: 6.558530807495117 ثانیه


In [15]:
r = np.linalg.matrix_rank(A_sparse)       ##چرا رتبه یک شده؟
r

1

from scipy.linalg import matrix_rank
rank = matrix_rank(A_sparse.toarray())
print("رتبه ماتریس:", rank)

In [27]:
start_time = time.time()

sol_ls_A_sparse = scipy.sparse.linalg.lsqr(A_sparse, dose_sparse_1d)

end_time = time.time()
total_execution_time = end_time - start_time
print(f"زمان کل اجرای کمترین مربعات ماتریس پراکنده\n: {total_execution_time} ثانیه")


زمان کل اجرای کمترین مربعات ماتریس پراکنده
: 0.2670164108276367 ثانیه


In [20]:

memory_usage_A_full = sys.getsizeof(A_full )
megabyte_size_A_full = memory_usage_A_full / 1024 / 1024

print(f"میزان حافظه A_full : {megabyte_size_A_full} مگابایت")

memory_usage_A_sparse = sys.getsizeof(A_sparse )
megabyte_size_A_sparse = memory_usage_A_sparse / 1024 / 1024

print(f"میزان حافظه A_sparse : {megabyte_size_A_sparse} مگابایت")

میزان حافظه A_full : 124.43244171142578 مگابایت
میزان حافظه A_sparse : 4.57763671875e-05 مگابایت


In [5]:
print(f'dose shape : {dose_sparse_1d.shape}')
print(f'A_sparse shape : {A_sparse.shape}')
#print(f'A_full shape : {A_full.shape}')

dose shape : (60183,)
A_sparse shape : (60183, 542)


In [24]:

memory_usage_sol_ls_A_sparse = sys.getsizeof(sol_ls_A_sparse )
megabyte_size_sol_ls_A_sparse = memory_usage_sol_ls_A_sparse / 1024 / 1024

print(f" sol_ls_A_sparse میزان حافظه :\n {megabyte_size_sol_ls_A_sparse} مگابایت")


 sol_ls_A_sparse میزان حافظه :
 0.00011444091796875 مگابایت


### تحلیل خروجی های مسئله ی کمترین مربعات خطی

In [25]:
# جواب بهینه
x_star = sol_ls_A_sparse[0]
print(f'x_star :\t {x_star[0:4]}')

#  تعداد مراحل اجرای الگوریتم
number_algorithm_execution_steps = sol_ls_A_sparse[2]
print(f'number_algorithm_execution_steps :\t {number_algorithm_execution_steps}')

#  نرم بردار سمت راست d
Rightـhandـsideـvectorـnormـd = sol_ls_A_sparse[3]
print(f'Rightـhandـsideـvectorـnormـd :\t {Rightـhandـsideـvectorـnormـd}')

#شاخص برای ارزیابی خروجی الگوریتم کمترین مربعات است	هرچقدر این نرم کوچکتر باشد نشانه بهتر بودن جواب بهینه است


#نرم ضرب داخلی ماتریس در جواب
l2_res = sol_ls_A_sparse[4]
print(f'L2 norm of residuals :\t{l2_res}')

#   مجموع مربعات خطای سیستم 
sum_of_square1 = sol_ls_A_sparse[5]
print(f'sum_of_square1 :\t {sum_of_square1}')

#   مجموع مربعات خطای ماتریس معکوس
sum_of_square2 = sol_ls_A_sparse[6]
print(f'sum_of_square2 :\t {sum_of_square2}')

# بردار صفر              ?????????????
Zeroـvector = sol_ls_A_sparse[9]
print(f'Zeroـvector :\t {Zeroـvector[0:24]}')

# کد بازگشتی 
Returnـcode = sol_ls_A_sparse[1]
print(f'Returnـcode :\t {Returnـcode}')

x_star :	 [ 6.52999050e+00  1.88266842e-01 -2.47236293e-05 -2.50110712e-05]
number_algorithm_execution_steps :	 39
Rightـhandـsideـvectorـnormـd :	 0.013811075528736654
L2 norm of residuals :	0.013811075528736654
sum_of_square1 :	 60.77704428103041
sum_of_square2 :	 82.64578136918476
Zeroـvector :	 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Returnـcode :	 1


####   کد بازگشتی=۱، نشان دهنده ی  موفقیت آمیز نبودن الگوریتم است. یعنی الگوریتم نتوانسته در تعداد مراحل مجاز مسئله را حل کند
####   لذا از این الگوریتم زیر به جای قبلی استفاده می شود

In [26]:
import numpy as np
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import lsmr

A = csc_matrix(A_sparse) 

# پارامترهای الگوریتم
tol = 1e-9
maxiter = 1000

# حل مسئله با LSMR
Sol_Lsmr= lsmr(A, dose_sparse_1d, damp=0.0, atol=tol, btol=tol, maxiter=maxiter)
   
print("Converged after %d iterations." % Sol_Lsmr[2])
print("Residual norm : %f" % Sol_Lsmr[3])
#print(f"Solution vector:\t {Sol_Lsmr[0]} ")
print(f'Returnـcode : ???\t{Sol_Lsmr[1]}')


Converged after 62 iterations.
Residual norm : 0.000016
Returnـcode : ???	1


In [5]:
beams = pp.Beams(data)
beams_full = pp.Beams(data, load_inf_matrix_full=True)
inf_matrix_full = pp.InfluenceMatrix(ct=ct, structs=structs, beams=beams_full, is_full=True)
plan_full = pp.Plan(ct, structs, beams, inf_matrix_full, clinical_criteria)
A_full = plan_full.inf_matrix.A

Creating BEV..
Loading full influence matrix..
Done


In [29]:
start_time = time.time()

sol_ls_A_full = scipy.sparse.linalg.lsqr(A_full, dose_sparse_1d)

end_time = time.time()
total_execution_time = end_time - start_time
print(f"زمان کل اجرای کمترین مربعات ماتریس کامل\n: {total_execution_time} ثانیه")



زمان کل اجرای کمترین مربعات ماتریس کامل
: 12.808384418487549 ثانیه


In [32]:

memory_usage_sol_ls_A_full = sys.getsizeof(sol_ls_A_full )
megabyte_size_sol_ls_A_full= memory_usage_sol_ls_A_full / 1024 / 1024

print(f" میزان حافظه حل کمترین مربعات ماتریس کامل :\n {megabyte_size_sol_ls_A_full} مگابایت")


 میزان حافظه حل کمترین مربعات ماتریس کامل :
 0.00011444091796875 مگابایت


In [105]:
# جواب بهینه
x_star = sol_ls_A_full[0]
print(f'x_star :\t {x_star[0:4]}')
   
#  تعداد مراحل اجرای الگوریتم
number_algorithm_execution_steps = sol_ls_A_full[2]
print(f'number_algorithm_execution_steps :\t {number_algorithm_execution_steps}')

#  نرم بردار سمت راست d
Rightـhandـsideـvectorـnormـd = sol_ls_A_full[3]
print(f'Rightـhandـsideـvectorـnormـd :\t {Rightـhandـsideـvectorـnormـd}')

#شاخص برای ارزیابی خروجی الگوریتم کمترین مربعات است	هرچقدر این نرم کوچکتر باشد نشانه بهتر بودن جواب بهینه است


#نرم ضرب داخلی ماتریس در جواب
l2_res = sol_ls_A_full[4]
print(f'L2 norm of residuals :\t {l2_res}')

#   مجموع مربعات خطای سیستم 
sum_of_square1 = sol_ls_A_full[5]
print(f'sum_of_square1 :\t {sum_of_square1}')

#   مجموع مربعات خطای ماتریس معکوس
sum_of_square2 = sol_ls_A_full[6]
print(f'sum_of_square2 :\t {sum_of_square2}')

# بردار صفر              ?????????????
Zeroـvector = sol_ls_A_full[9]
print(f'Zeroـvector :\t{Zeroـvector[0:24]}')

# کد بازگشتی 
Returnـcode = sol_ls_A_full[1]
print(f'Returnـcode :\t{Returnـcode}')

x_star :	 [-2.54878706 -2.03016544 -1.92035387 -2.45389775]
number_algorithm_execution_steps :	 42
Rightـhandـsideـvectorـnormـd :	 267.8564459470595
L2 norm of residuals :	 267.8564459470595
sum_of_square1 :	 68.28164415912218
sum_of_square2 :	 89.6008917445037
Zeroـvector :	[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Returnـcode :	2


In [33]:
#Number of non-zero entries, equivalent to
print(f'ابعاد ماتریس کامل : {A_full.ndim}')
nonzero_count = np.count_nonzero(A_full)
print("تعداد مقادیر غیر صفر ماتریس کامل: ", nonzero_count)
print("تعداد مقادیر غیر صفر ماتریس پراکنده: ", A_sparse.getnnz())

ابعاد ماتریس کامل : 2
تعداد مقادیر غیر صفر ماتریس کامل:  10279072
تعداد مقادیر غیر صفر ماتریس پراکنده:  335573


In [34]:
## بررسی  پراکندگی ماتریس

is_sparse = sparse.issparse(A_sparse)
is_sparse

True

In [35]:
## بررسی پراکندگی  ماتریس

is_sparse = sparse.issparse(A_full)
is_sparse

False

In [36]:
A_sparse.format

'csr'

In [37]:
A_sparse.getmaxprint         #Maximum number of elements to display when printed.

<bound method spmatrix.getmaxprint of <60183x542 sparse matrix of type '<class 'numpy.float32'>'
	with 335573 stored elements in Compressed Sparse Row format>>

In [38]:
A_sparse.nonzero()    #nonzero indices

(array([ 9917,  9918,  9919, ..., 50949, 50950, 51002]),
 array([233, 233, 233, ..., 151, 152, 153]))